# 2. Neural Style Transfer on your local machine

To make sure all the scripts that are working correctly, we need to test them on our local machine first. Specifically, we'll be testing the components that will be in our __flask app__ and our __scoring app__.

### Flask App
The __flask app__ is responsible for:
1. preprocessing the images into individual frames and extracting the audio file
2. adding references of the individual images to the Service Bus queue
3. waiting until style transfer has been applied to all images
4. postprocessing the style transfer images by stitching them together into a video and re-adding the audio clip

The __flask app__ contains the following files:
- `preprocess.py` 
- `add_images_to_queue.py` 
- `postprocess.py` 
- `util.py`
- `main.py`

`preprocess.py`, `add_images_to_queue.py`, and `postprocess.py` can be executed independently. This is to allow us to test each piece. The `main.py` file will call functions the aforementioned files. It will start by preprocessing the video then adding images to the service bus queue. Then it will wait for the messages in the queue to be dequeued and processed by another running process. (This occurs in the __scoring app__.) After all the images in the queue are processed, the `main.py` will start to postprocess the output frames, and recreate a new video, but this time with the style transfered frames.

### Scoring App
The __scoring app__ is responsible for:
1. polling the Service Bus queue for new images 
2. applying style transfer to those images and saving them back to storage

The __scoring app__ containers the following files:
- `main.py`
- `process_images_from_queue.py`
- `style_transfer.py`
- `util.py`

`main.py` starts the app by calling on the function in the `process_images_from_queue.py` file. This is responsible for continuously polling the service bus queue and checking for new messages. As images get pulled off the queue, the process will then call the stylize function in the `style_transfer.py` file which performs the actual style transfer. The generated style images are saved back to blob directly from this process. 

---

### Import package and load .env

In [3]:
from dotenv import set_key, get_key, find_dotenv, load_dotenv
from pathlib import Path
import json
import os

In [4]:
env_path = find_dotenv(raise_error_if_not_found=True)
load_dotenv(env_path)

True

### Preprocess video 
Run the preprocessing step on the video `orangutan.mp4` that is in blob. Specify the video (including the extention) and the mounted directory to access blob. 

In [5]:
!python flask_app/preprocess.py \
    --video orangutan.mp4 \
    --storage-mount-dir data

ffmpeg version 4.0 Copyright (c) 2000-2018 the FFmpeg developers
  built with gcc 7.2.0 (crosstool-NG fa8859cb)
  configuration: --prefix=/home/aperture/anaconda3/envs/bc_aks --cc=/opt/conda/conda-bld/ffmpeg_1531088893642/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --enable-shared --enable-static --enable-zlib --enable-pic --enable-gpl --enable-version3 --disable-nonfree --enable-hardcoded-tables --enable-avresample --enable-libfreetype --disable-openssl --disable-gnutls --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --disable-libx264
  libavutil      56. 14.100 / 56. 14.100
  libavcodec     58. 18.100 / 58. 18.100
  libavformat    58. 12.100 / 58. 12.100
  libavdevice    58.  3.100 / 58.  3.100
  libavfilter     7. 16.100 /  7. 16.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  1.100 /  5.  1.100
  libswresample   3.  1.100 /  3.  1.100
  libpostproc    55.  1.100 / 55.  1.100
Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'data/orangu

### Add messages to queue
The preprocessing step splits the video into individual frames. Now add those frames to the queue to be processed later.

In [6]:
!python flask_app/add_images_to_queue.py \
    --video-name orangutan \
    --storage-mount-dir data

### Run the main style transfer process
The `scoring_app/main.py` file will dequeue images from the queue and apply style transfer to the images.

In [7]:
!python scoring_app/main.py --terminate

2019-07-15 08:17:04,301 [root:process_images_from_queue.py:34] DEBUG - Start listening to queue 'oxfordqueue' on service bus...
2019-07-15 08:17:04,301 [root:process_images_from_queue.py:39] DEBUG - Peek queue...
2019-07-15 08:17:04,377 [root:process_images_from_queue.py:75] DEBUG - Queue message body: {'input_frame': '000823_frame.jpg', 'video_name': 'orangutan'}
2019-07-15 08:17:04,377 [root:process_images_from_queue.py:78] DEBUG - Starting style transfer on data/orangutan/input_frames/000823_frame.jpg
2019-07-15 08:17:04,434 [root:style_transfer.py:145] DEBUG - Processing data/orangutan/input_frames/000823_frame.jpg
2019-07-15 08:17:10,318 [root:process_images_from_queue.py:87] DEBUG - Finished style transfer on data/orangutan/input_frames/000823_frame.jpg
2019-07-15 08:17:10,319 [root:process_images_from_queue.py:90] DEBUG - Deleting queue message...
2019-07-15 08:17:10,363 [root:process_images_from_queue.py:39] DEBUG - Peek queue...
2019-07-15 08:17:10,387 [root:process_images_fro

2019-07-15 08:18:08,305 [root:process_images_from_queue.py:39] DEBUG - Peek queue...
2019-07-15 08:18:08,319 [root:process_images_from_queue.py:75] DEBUG - Queue message body: {'input_frame': '000812_frame.jpg', 'video_name': 'orangutan'}
2019-07-15 08:18:08,319 [root:process_images_from_queue.py:78] DEBUG - Starting style transfer on data/orangutan/input_frames/000812_frame.jpg
2019-07-15 08:18:08,350 [root:style_transfer.py:145] DEBUG - Processing data/orangutan/input_frames/000812_frame.jpg
2019-07-15 08:18:14,010 [root:process_images_from_queue.py:87] DEBUG - Finished style transfer on data/orangutan/input_frames/000812_frame.jpg
2019-07-15 08:18:14,011 [root:process_images_from_queue.py:90] DEBUG - Deleting queue message...
2019-07-15 08:18:14,138 [root:process_images_from_queue.py:39] DEBUG - Peek queue...
2019-07-15 08:18:14,152 [root:process_images_from_queue.py:75] DEBUG - Queue message body: {'input_frame': '000811_frame.jpg', 'video_name': 'orangutan'}
2019-07-15 08:18:14,15

2019-07-15 08:19:12,529 [root:style_transfer.py:145] DEBUG - Processing data/orangutan/input_frames/000801_frame.jpg
2019-07-15 08:19:18,862 [root:process_images_from_queue.py:87] DEBUG - Finished style transfer on data/orangutan/input_frames/000801_frame.jpg
2019-07-15 08:19:18,863 [root:process_images_from_queue.py:90] DEBUG - Deleting queue message...
2019-07-15 08:19:19,006 [root:process_images_from_queue.py:39] DEBUG - Peek queue...
2019-07-15 08:19:19,020 [root:process_images_from_queue.py:75] DEBUG - Queue message body: {'input_frame': '000800_frame.jpg', 'video_name': 'orangutan'}
2019-07-15 08:19:19,020 [root:process_images_from_queue.py:78] DEBUG - Starting style transfer on data/orangutan/input_frames/000800_frame.jpg
2019-07-15 08:19:19,053 [root:style_transfer.py:145] DEBUG - Processing data/orangutan/input_frames/000800_frame.jpg
2019-07-15 08:19:24,713 [root:process_images_from_queue.py:87] DEBUG - Finished style transfer on data/orangutan/input_frames/000800_frame.jpg
2

2019-07-15 08:20:22,554 [root:process_images_from_queue.py:87] DEBUG - Finished style transfer on data/orangutan/input_frames/000790_frame.jpg
2019-07-15 08:20:22,554 [root:process_images_from_queue.py:90] DEBUG - Deleting queue message...
2019-07-15 08:20:22,601 [root:process_images_from_queue.py:39] DEBUG - Peek queue...
2019-07-15 08:20:22,617 [root:process_images_from_queue.py:75] DEBUG - Queue message body: {'input_frame': '000789_frame.jpg', 'video_name': 'orangutan'}
2019-07-15 08:20:22,617 [root:process_images_from_queue.py:78] DEBUG - Starting style transfer on data/orangutan/input_frames/000789_frame.jpg
2019-07-15 08:20:22,650 [root:style_transfer.py:145] DEBUG - Processing data/orangutan/input_frames/000789_frame.jpg
2019-07-15 08:20:28,614 [root:process_images_from_queue.py:87] DEBUG - Finished style transfer on data/orangutan/input_frames/000789_frame.jpg
2019-07-15 08:20:28,614 [root:process_images_from_queue.py:90] DEBUG - Deleting queue message...
2019-07-15 08:20:28,6

2019-07-15 08:21:26,626 [root:process_images_from_queue.py:39] DEBUG - Peek queue...
2019-07-15 08:21:26,640 [root:process_images_from_queue.py:75] DEBUG - Queue message body: {'input_frame': '000778_frame.jpg', 'video_name': 'orangutan'}
2019-07-15 08:21:26,640 [root:process_images_from_queue.py:78] DEBUG - Starting style transfer on data/orangutan/input_frames/000778_frame.jpg
2019-07-15 08:21:26,672 [root:style_transfer.py:145] DEBUG - Processing data/orangutan/input_frames/000778_frame.jpg
2019-07-15 08:21:32,200 [root:process_images_from_queue.py:87] DEBUG - Finished style transfer on data/orangutan/input_frames/000778_frame.jpg
2019-07-15 08:21:32,200 [root:process_images_from_queue.py:90] DEBUG - Deleting queue message...
2019-07-15 08:21:32,301 [root:process_images_from_queue.py:39] DEBUG - Peek queue...
2019-07-15 08:21:32,316 [root:process_images_from_queue.py:75] DEBUG - Queue message body: {'input_frame': '000777_frame.jpg', 'video_name': 'orangutan'}
2019-07-15 08:21:32,31

...

2019-07-15 09:34:58,874 [root:process_images_from_queue.py:87] DEBUG - Finished style transfer on data/orangutan/input_frames/000055_frame.jpg
2019-07-15 09:34:58,874 [root:process_images_from_queue.py:90] DEBUG - Deleting queue message...
2019-07-15 09:34:58,945 [root:process_images_from_queue.py:39] DEBUG - Peek queue...
2019-07-15 09:34:58,959 [root:process_images_from_queue.py:75] DEBUG - Queue message body: {'input_frame': '000054_frame.jpg', 'video_name': 'orangutan'}
2019-07-15 09:34:58,960 [root:process_images_from_queue.py:78] DEBUG - Starting style transfer on data/orangutan/input_frames/000054_frame.jpg
2019-07-15 09:34:58,990 [root:style_transfer.py:145] DEBUG - Processing data/orangutan/input_frames/000054_frame.jpg
2019-07-15 09:35:04,780 [root:process_images_from_queue.py:87] DEBUG - Finished style transfer on data/orangutan/input_frames/000054_frame.jpg
2019-07-15 09:35:04,780 [root:process_images_from_queue.py:90] DEBUG - Deleting queue message...
2019-07-15 09:35:04,8

2019-07-15 09:36:04,653 [root:process_images_from_queue.py:39] DEBUG - Peek queue...
2019-07-15 09:36:04,668 [root:process_images_from_queue.py:75] DEBUG - Queue message body: {'input_frame': '000043_frame.jpg', 'video_name': 'orangutan'}
2019-07-15 09:36:04,668 [root:process_images_from_queue.py:78] DEBUG - Starting style transfer on data/orangutan/input_frames/000043_frame.jpg
2019-07-15 09:36:04,703 [root:style_transfer.py:145] DEBUG - Processing data/orangutan/input_frames/000043_frame.jpg
2019-07-15 09:36:10,720 [root:process_images_from_queue.py:87] DEBUG - Finished style transfer on data/orangutan/input_frames/000043_frame.jpg
2019-07-15 09:36:10,721 [root:process_images_from_queue.py:90] DEBUG - Deleting queue message...
2019-07-15 09:36:10,765 [root:process_images_from_queue.py:39] DEBUG - Peek queue...
2019-07-15 09:36:10,786 [root:process_images_from_queue.py:75] DEBUG - Queue message body: {'input_frame': '000042_frame.jpg', 'video_name': 'orangutan'}
2019-07-15 09:36:10,78

2019-07-15 09:37:11,663 [root:style_transfer.py:145] DEBUG - Processing data/orangutan/input_frames/000032_frame.jpg
2019-07-15 09:37:17,627 [root:process_images_from_queue.py:87] DEBUG - Finished style transfer on data/orangutan/input_frames/000032_frame.jpg
2019-07-15 09:37:17,627 [root:process_images_from_queue.py:90] DEBUG - Deleting queue message...
2019-07-15 09:37:17,672 [root:process_images_from_queue.py:39] DEBUG - Peek queue...
2019-07-15 09:37:17,687 [root:process_images_from_queue.py:75] DEBUG - Queue message body: {'input_frame': '000031_frame.jpg', 'video_name': 'orangutan'}
2019-07-15 09:37:17,687 [root:process_images_from_queue.py:78] DEBUG - Starting style transfer on data/orangutan/input_frames/000031_frame.jpg
2019-07-15 09:37:17,722 [root:style_transfer.py:145] DEBUG - Processing data/orangutan/input_frames/000031_frame.jpg
2019-07-15 09:37:23,788 [root:process_images_from_queue.py:87] DEBUG - Finished style transfer on data/orangutan/input_frames/000031_frame.jpg
2

2019-07-15 09:38:23,890 [root:process_images_from_queue.py:87] DEBUG - Finished style transfer on data/orangutan/input_frames/000021_frame.jpg
2019-07-15 09:38:23,891 [root:process_images_from_queue.py:90] DEBUG - Deleting queue message...
2019-07-15 09:38:24,003 [root:process_images_from_queue.py:39] DEBUG - Peek queue...
2019-07-15 09:38:24,019 [root:process_images_from_queue.py:75] DEBUG - Queue message body: {'input_frame': '000020_frame.jpg', 'video_name': 'orangutan'}
2019-07-15 09:38:24,019 [root:process_images_from_queue.py:78] DEBUG - Starting style transfer on data/orangutan/input_frames/000020_frame.jpg
2019-07-15 09:38:24,058 [root:style_transfer.py:145] DEBUG - Processing data/orangutan/input_frames/000020_frame.jpg
2019-07-15 09:38:30,103 [root:process_images_from_queue.py:87] DEBUG - Finished style transfer on data/orangutan/input_frames/000020_frame.jpg
2019-07-15 09:38:30,104 [root:process_images_from_queue.py:90] DEBUG - Deleting queue message...
2019-07-15 09:38:30,1

2019-07-15 09:39:30,791 [root:process_images_from_queue.py:39] DEBUG - Peek queue...
2019-07-15 09:39:30,806 [root:process_images_from_queue.py:75] DEBUG - Queue message body: {'input_frame': '000009_frame.jpg', 'video_name': 'orangutan'}
2019-07-15 09:39:30,806 [root:process_images_from_queue.py:78] DEBUG - Starting style transfer on data/orangutan/input_frames/000009_frame.jpg
2019-07-15 09:39:30,843 [root:style_transfer.py:145] DEBUG - Processing data/orangutan/input_frames/000009_frame.jpg
2019-07-15 09:39:36,706 [root:process_images_from_queue.py:87] DEBUG - Finished style transfer on data/orangutan/input_frames/000009_frame.jpg
2019-07-15 09:39:36,706 [root:process_images_from_queue.py:90] DEBUG - Deleting queue message...
2019-07-15 09:39:36,777 [root:process_images_from_queue.py:39] DEBUG - Peek queue...
2019-07-15 09:39:36,792 [root:process_images_from_queue.py:75] DEBUG - Queue message body: {'input_frame': '000008_frame.jpg', 'video_name': 'orangutan'}
2019-07-15 09:39:36,79

### Postprocess the frames
Now that the frames have been processed, we can stitch then back together and reattach the audio file using the `flask_app/postprocess.py` file.

In [8]:
!python flask_app/postprocess.py \
    --video-name orangutan \
    --storage-mount-dir data

ffmpeg version 4.0 Copyright (c) 2000-2018 the FFmpeg developers
  built with gcc 7.2.0 (crosstool-NG fa8859cb)
  configuration: --prefix=/home/aperture/anaconda3/envs/bc_aks --cc=/opt/conda/conda-bld/ffmpeg_1531088893642/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --enable-shared --enable-static --enable-zlib --enable-pic --enable-gpl --enable-version3 --disable-nonfree --enable-hardcoded-tables --enable-avresample --enable-libfreetype --disable-openssl --disable-gnutls --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --disable-libx264
  libavutil      56. 14.100 / 56. 14.100
  libavcodec     58. 18.100 / 58. 18.100
  libavformat    58. 12.100 / 58. 12.100
  libavdevice    58.  3.100 / 58.  3.100
  libavfilter     7. 16.100 /  7. 16.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  1.100 /  5.  1.100
  libswresample   3.  1.100 /  3.  1.100
  libpostproc    55.  1.100 / 55.  1.100
Input #0, image2, from 'data/orangutan/output_frames

Play the video! It will be called `{video_name}_processed.mp4`

In [9]:
%%HTML
<video width="320" height="240" controls>
  <source src="data/orangutan/orangutan_processed.mp4" type="video/mp4">
</video>

### Additional commands to monitor
Use these additional commands to monitor.

```bash
# checkout how many elements are in the queue
!az servicebus queue show \
    --name {get_key(env_path, "SB_QUEUE")} \
    --namespace-name {get_key(env_path, "SB_NAMESPACE")} \
    --resource-group {get_key(env_path, "RESOURCE_GROUP")} \
    --query "countDetails.activeMessageCount"
```

---

Continue to the next [notebook](/notebooks/03_create_aks_cluster.ipynb).